In [ ]:
import numpy as np
import transformers
import torch
import transformers as tf
import shutil
import urllib.request
import tarfile
from transformers import BertModel, BertTokenizer
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Load các pretrain model
model = BertModel.from_pretrained('bert-base-uncased')
# Load model tu BERT pretrain. Cu the la model bert-base (bản nhỏ) và uncased (không care viết hoa, thường)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# load cái Tokenizer. Sử dụng để tokenize các từ trong câu.

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# Hàm chuẩn hoá câu
def standardize_data(row):
    # Xóa dấu chấm, phẩy, hỏi ở cuối câu
    row = re.sub(r"[\.,\?]+$-", "", row)
    # Xóa tất cả dấu chấm, phẩy, chấm phẩy, chấm thang, ... trong câu
    row = row.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace(""", " ") \
        .replace(":", " ").replace(""", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("-", " ").replace("?", " ") \
        .replace("<br />", " ").replace("?", " ")
    row = row.strip().lower()
    return row

In [ ]:
def generate_batches(text, label, batch_size):
    for i in range(0, len(text), batch_size):
        yield text[i:i+batch_size], label[i:i+batch_size]

In [ ]:
def load_data(batch_size):
    v_text = []
    v_label = []

    with open('/content/gdrive/MyDrive/Deep_Learning/test125.csv', encoding='latin-1') as f:
        lines = f.readlines()
        lines = lines[:1999]

    for line in lines:
        line = line.replace("\n","")
        v_text.append(standardize_data(line[:128] + line[-382:]))
        v_label.append(int(line[-1:].replace("\n", "")))

    print(v_label)
    return v_text, v_label

In [ ]:
def make_bert_features(v_text, batch_size):
    v_tokenized = []
    max_len = 512

    # Tokenize các câu và thêm padding
    for i_text in v_text:
        line = tokenizer.encode(i_text, max_length=max_len, truncation=True, padding='max_length')
        v_tokenized.append(line)

    # Chuyển thành tensor
    input_ids = torch.tensor(v_tokenized).to(torch.long)
    attention_mask = torch.where(input_ids == tokenizer.pad_token_id, 0, 1)

    # Lấy features đầu ra từ BERT
    with torch.no_grad():
        last_hidden_states = model(input_ids=input_ids, attention_mask=attention_mask)[0]

    v_features = last_hidden_states[:, 0, :].numpy()
    return v_features

In [ ]:
import re
re.compile('<title>(.*)</title>')

re.compile(r'<title>(.*)</title>', re.UNICODE)

In [ ]:
batch_size=4
print("Chuẩn bị load dữ liệu....")
text, label = load_data(batch_size)
print("Đã load dữ liệu xong")

Chuẩn bị load dữ liệu....
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
print("Chuẩn bị tạo features từ BERT.....")
batch_generator = generate_batches(text, label, batch_size)
all_features = []
for batch_text, batch_label in batch_generator:
    batch_features = make_bert_features(batch_text, batch_size)
    #print(f"Processing batch {i} out of {len(batch_generator)}")
    all_features.append(batch_features)
features = np.concatenate(all_features, axis=0)

Chuẩn bị tạo features từ BERT.....


In [ ]:
from sklearn.model_selection import train_test_split
X_train_val, X_test, y_train_val, y_test = train_test_split(features, label, test_size=0.3, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.3, random_state=0)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model = load_model('/content/drive/MyDrive/Deep_Learning/finetuned_imdb_model1.keras')

In [ ]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=2e-5)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
y_train=to_categorical(y_train, num_classes=2)
y_val=to_categorical(y_val, num_classes=2)
y_test=to_categorical(y_test, num_classes=2)

In [ ]:
model.fit(X_train, y_train, epochs=1400, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/1400
31/31 [==============================] - 2s 20ms/step - loss: 0.3189 - accuracy: 0.9356 - val_loss: 0.3156 - val_accuracy: 0.9214
Epoch 2/1400
31/31 [==============================] - 0s 7ms/step - loss: 0.3175 - accuracy: 0.9377 - val_loss: 0.3143 - val_accuracy: 0.9238
Epoch 3/1400
31/31 [==============================] - 0s 8ms/step - loss: 0.3162 - accuracy: 0.9377 - val_loss: 0.3131 - val_accuracy: 0.9238
Epoch 4/1400
31/31 [==============================] - 0s 8ms/step - loss: 0.3149 - accuracy: 0.9377 - val_loss: 0.3120 - val_accuracy: 0.9238
Epoch 5/1400
31/31 [==============================] - 0s 9ms/step - loss: 0.3136 - accuracy: 0.9397 - val_loss: 0.3107 - val_accuracy: 0.9262
Epoch 6/1400
31/31 [==============================] - 0s 5ms/step - loss: 0.3124 - accuracy: 0.9418 - val_loss: 0.3096 - val_accuracy: 0.9286
Epoch 7/1400
31/31 [==============================] - 0s 6ms/step - loss: 0.3111 - accuracy: 0.9428 - val_loss: 0.3084 - val_accuracy: 0.9286
Epoch

In [ ]:
score = model.evaluate(X_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

19/19 [==============================] - 0s 2ms/step - loss: 0.0468 - accuracy: 0.9950
Test loss: 0.04681287705898285
Test accuracy: 0.9950000047683716


In [ ]:
model.save('finetuned_imdb_model21.keras')